## Init

In [1]:
# from urllib.parse import quote
import pandas as pd
import os
import sys
import glob
import json

# my packages
from dData.dData import dData

# utilities
plot = pd.DataFrame
from itertools import combinations

from bs4 import BeautifulSoup

# import all module in order to use reload later on the paclet
import importlib
import toolsGeneral.main
import toolsOSM.overpass
import tools.filters

# use this line to ensure intellisense recognizes new symbols
from toolsGeneral.main import *
from toolsOSM.overpass import *
from tools.filters import *

def reloadPckg():
    importlib.reload(toolsGeneral.main)
    importlib.reload(tools.filters)
    globals().update({
        name: getattr(toolsGeneral.main, name) 
        for name 
        in dir(toolsGeneral.main) 
        if not name.startswith('_')
    })
    globals().update({
        name: getattr(tools.filters, name) 
        for name 
        in dir(tools.filters) 
        if not name.startswith('_')
    })
    # global dump
    # dump = toolsGeneral.main.dump



In [2]:
osmMetaCountrDict = load(r"C:\Users\gonta\D\code\development\OSM\get countries ids and add labels\data\osmMetaCountrDict.json")

In [3]:
cntrName = "Germany"
cntrRaw = f"./data/{osmMetaCountrDict[cntrName]['id']}/rawOSMRes.json"
cntrDataDir = f"./data/{osmMetaCountrDict[cntrName]['id']}/"
cntrId = osmMetaCountrDict[cntrName]['id']
cntrId

'51477'

# collect

## make overpass query and dump

In [ ]:
rawOSMRes = getOSMIDAddsStruct(cntrId, [4,6,8])
list(rawOSMRes.keys())

# rawOSMRes = getOSMAddsTRecursedown(cntrId, [4,6,8])
# list(rawOSMRes.keys())

In [26]:
dump(cntrRaw, rawOSMRes)

# sanitize

In [109]:
rawElems = load(cntrRaw)['elements']
print(len(rawElems))

11762


## test

In [110]:
[miss, dup, relLeaks] = testSanitize(rawElems)

missing name:  0
duplicate elements:  9
relations from other countries:  494


## modify

### make rules

In [119]:
modify = load('./data/modifications.json')

#### to delete

In [111]:
relLeaksIds = [str(ele['id']) for ele in relLeaks]
print(len([ele['id'] for ele in relLeaks]))
print(len(deleteDuplicates(relLeaksIds)))

494
492


In [93]:
relLeaksChilds = [ele for ele in rawElems if ele['tags'].get('parent_id') in relLeaksIds]
len(relLeaksChilds)

492

In [116]:
toDelete = [*relLeaksChilds, *relLeaks]
print(len(toDelete))
toDelete = deleteDuplicates(toDelete)
print(len(toDelete))

986
496


In [123]:
[{'id': str(ele['id']), 'parent_id':ele.get('parent_id',None)} for ele in toDelete]

[{'id': '62485', 'parent_id': None},
 {'id': '442314', 'parent_id': None},
 {'id': '442452', 'parent_id': None},
 {'id': '436593', 'parent_id': None},
 {'id': '436596', 'parent_id': None},
 {'id': '436597', 'parent_id': None},
 {'id': '436598', 'parent_id': None},
 {'id': '436947', 'parent_id': None},
 {'id': '436948', 'parent_id': None},
 {'id': '436949', 'parent_id': None},
 {'id': '436950', 'parent_id': None},
 {'id': '436954', 'parent_id': None},
 {'id': '436955', 'parent_id': None},
 {'id': '436956', 'parent_id': None},
 {'id': '436957', 'parent_id': None},
 {'id': '436958', 'parent_id': None},
 {'id': '436959', 'parent_id': None},
 {'id': '436960', 'parent_id': None},
 {'id': '436961', 'parent_id': None},
 {'id': '436962', 'parent_id': None},
 {'id': '436963', 'parent_id': None},
 {'id': '436967', 'parent_id': None},
 {'id': '436968', 'parent_id': None},
 {'id': '438942', 'parent_id': None},
 {'id': '438943', 'parent_id': None},
 {'id': '438944', 'parent_id': None},
 {'id': '4389

In [121]:
modify[cntrId]['delete']

[{'id': '3659532', 'parent_id': '62372'},
 {'id': '3659532', 'parent_id': '62341'},
 {'id': '62485', 'parent_id': '435516'},
 {'id': '442452', 'parent_id': '62467'},
 {'id': '1115033', 'parent_id': '62401'},
 {'id': '1115034', 'parent_id': '62586'},
 {'id': '1808583', 'parent_id': '62597'},
 {'id': '439554', 'parent_id': '62485'},
 {'id': '534534', 'parent_id': '442452'},
 {'id': '3678366', 'parent_id': '1739379'}]

#### to modify

In [186]:
id = 3678366
temp = [ele['tags']['parent_id'] for ele in rawOSMRes['elements'] if ele['id'] == id]
temp.append(str(id))
print(','.join(temp))
temp = list(map(lambda x: f'https://www.openstreetmap.org/relation/{x}', temp))
temp

1431517,1739379,3678366


['https://www.openstreetmap.org/relation/1431517',
 'https://www.openstreetmap.org/relation/1739379',
 'https://www.openstreetmap.org/relation/3678366']

In [ ]:
temp = {

}

In [68]:
removeEnts = {**removeEnts, **temp}

In [ ]:
dump('./data/modifications.json', removeEnts)

### apply

In [76]:
modifications = load('./data/modifications.json')

In [113]:
import copy
rawElemsTSane = pd.DataFrame(copy.deepcopy(rawElems))
len(rawElemsTSane.index)

11762

In [114]:
delete = deleteDuplicates(pd.DataFrame(relLeaks)['id'].to_list())
print(len(delete))
rawElemsTSane = rawElemsTSane[~rawElemsTSane['id'].isin(delete)]
rawElemsTSane = rawElemsTSane.to_dict('records')
print(len(rawElemsTSane))

3
11758


In [115]:
def applyChanges(lis, changes):

    toDelete = [[change['id'], change.get('parent_id','None')] for change in changes['delete']]

    def pattern(ele): 
        return [str(ele['id']), ele['tags']['parent_id'] if 'parent_id' in ele['tags'] else 'None']

    return  [ele for ele in lis if pattern(ele) not in toDelete]

In [116]:
print(len(rawElemsTSane))
changes = modifications[cntrId]
print(len(changes['delete']))
rawElemsTSane = applyChanges(rawElemsTSane, changes)
print(len(rawElemsTSane))

11758
10
11749


In [86]:
changes

{'delete': [{'id': '3659532', 'parent_id': '62372'},
  {'id': '3659532', 'parent_id': '62341'},
  {'id': '62485', 'parent_id': '435516'},
  {'id': '442452', 'parent_id': '62467'},
  {'id': '1115033', 'parent_id': '62401'},
  {'id': '1115034', 'parent_id': '62586'},
  {'id': '1808583', 'parent_id': '62597'},
  {'id': '439554', 'parent_id': '62485'},
  {'id': '534534', 'parent_id': '442452'},
  {'id': '3678366', 'parent_id': '1739379'}],
 'modify': [{'id': '3678366', 'parent_id': '28322', 'admin_level': '6'}]}

In [189]:
testSanitize(rawOSMResTSane)

missing name:  0
duplicate elements:  0


[[], []]

In [190]:
[ele for ele in rawOSMResTSane['elements'] if ele['id'] == 47826]

[]

# normalize

#### make relsDataIndex

In [191]:
relsDataIndex = {str(x["id"]):x for x in rawOSMResTSane["elements"]}
# add parent_id to keys on first level
for id, value in relsDataIndex.items():
    if value["tags"]["admin_level"] == '2':
        value["parent_id"] = "#"
    elif value["tags"]["admin_level"] == osmMetaCountrDict[cntrName]["addLvlsNum"][0]:
        value["parent_id"] = cntrId
    elif value["tags"]["admin_level"] in osmMetaCountrDict[cntrName]["addLvlsNum"][1:]:
        value["parent_id"] = value["tags"]["parent_id"]      
        # del value["parent_id"]

    value["id"] = str(id)
    # del value["id"]
    relsDataIndex[id] = sortDictKeys(value, ["type", "id", "parent_id", "tags"])

len(list(relsDataIndex.keys()))

11752

#### make childsIndex

In [192]:
idPairs = {key:value["parent_id"] for key, value in relsDataIndex.items() if value["parent_id"] != "#"} 
# creates an empty dictionary
childsIndex = {}
for k, v in idPairs.items():
    # if key doesn't exist then make list else append to list
    childsIndex.setdefault(v, list()).append(str(k))
    
len(childsIndex)

312

In [193]:
dump(os.path.join(cntrDataDir, "childsIndex.json"), childsIndex)

In [194]:
# Add childs to rels data index
for id, value in relsDataIndex.items():
    relsDataIndex[id] = sortDictKeys({"childs":childsIndex.get(id, []),**relsDataIndex[id]}, ["type", "id", "parent_id", "childs","tags"])

In [195]:
dump(os.path.join(cntrDataDir, "relsDataIndex.json"), relsDataIndex)

#### [OLD] make data tree and min tree 

In [81]:
# make tags properties to use in selection
relTagsIndex = { k : v["tags"] for k, v in relsDataIndex.items()}

In [ ]:
indexKeys = list(childsIndex.keys())

def makeAddsTree(idList, dataIndex):
    return {id: {**dataIndex[id], "childs":(makeAddsTree(relsDataIndex[id]["childs"], dataIndex) if id in indexKeys else {})} for id in idList }

In [29]:
keysMin = ["name"]
keys = ["id", "name", "official_name", "parent_id", "ISO3166-2", "admin_level"]
addsDataTree = makeAddsTree([countryID], {k:dictFilterKeys(v, keysMin) for k, v in relTagsIndex.items()})
len(addsDataTree[countryID]["childs"])

17

In [30]:
dump(os.path.join(outDir, "addsDataTree.json"), addsDataTree)

# validate

In [10]:
relsDataIndex = load(os.path.join(cntrDataDir, "relsDataIndex.json"))
print(len(relsDataIndex))
childsIndex = load(os.path.join(cntrDataDir, "childsIndex.json"))
print(len(childsIndex))

11752
312


In [15]:
print(len(relsDataIndex))
relsDataIndexTSane = copy.deepcopy(relsDataIndex)
# 1. Check leak of divisions from other countries
test1 = test('leaksFromOtherCountries')(relsDataIndexTSane, cntrId)
# 2. Intersection between childs
print('Intersection between childs?: ', len(findIntersection(list(childsIndex.values()))))

11752
Entities that not belong to the country:  3
Intersection between childs?:  0


In [12]:
for ele in list(test1.keys()):
    relsDataIndexTSane.pop(ele)
len(list(relsDataIndexTSane.keys()))

11749

In [150]:
test('leaksFromOtherCountries')(relsDataIndexTSane, cntrId)

Entities that not belong to the country:  0


{}

In [201]:
relsDataIndex['47826']

{'type': 'relation',
 'id': '47826',
 'parent_id': '51477',
 'childs': [],
 'tags': {'ISO3166-2': 'NL-GR',
  'admin_level': '4',
  'boundary': 'administrative',
  'designation': 'provincie',
  'flag': 'File:Flag_Groningen.svg',
  'name': 'Groningen',
  'name:ar': 'خرونينغن',
  'name:de': 'Groningen',
  'name:en': 'Groningen',
  'name:eo': 'Groningo',
  'name:es': 'Groninga',
  'name:fr': 'Groningue',
  'name:fy': 'Grinslân',
  'name:hu': 'Groningen',
  'name:it': 'Groninga',
  'name:ja': 'フローニンゲン州',
  'name:la': 'Groninga',
  'name:nds': 'Grunnen',
  'name:nl': 'Groningen',
  'name:pcd': 'Groningue',
  'name:pt': 'Groninga',
  'name:ro': 'Groningen',
  'name:ru': 'Гронинген',
  'name:sr': 'Гронинген',
  'name:tok': 'ma Konine',
  'name:uk': 'Гронінген',
  'name:vls': 'Groniengn',
  'name:zh-Hans': '格罗宁根',
  'name:zh-Hant': '格羅寧根',
  'note:ISO3166-2': 'Groningen is a subset (type Province) of the Kingdom of the Netherlands (ISO3166-1 country code NL)',
  'official_name': 'Groningen',
  

In [200]:
relsDataIndexTSane['47826']

KeyError: '47826'

In [151]:
dump(f'./data/{cntrId}/relsDataIndex.json', relsDataIndexTSane)

# review

In [124]:
relsDataIndex = load(os.path.join(cntrDataDir, "relsDataIndex.json"))
print(len(relsDataIndex))
childsIndex = load(os.path.join(cntrDataDir, "childsIndex.json"))
print(len(childsIndex))

11749
312


In [125]:
lvl1 = [v["id"] for v in relsDataIndex.values() if v["tags"]["admin_level"] == '4']
print(len(lvl1))
lvl2 = [v["id"] for v in relsDataIndex.values() if v["tags"]["admin_level"] == '6']
print(len(lvl2))
lvl3 = [v["id"] for v in relsDataIndex.values() if v["tags"]["admin_level"] == '8']
print(len(lvl3))

17
398
11333


In [114]:
getKeys = ["id", "name", "official_name", "parent_id", "ISO3166-2", "admin_level","childs_num"]

### lvl 1

In [126]:
props = [{**dictRemoveKeys(relsDataIndex[id],["childs", "tags"]), **relsDataIndex[id]["tags"], "childs_num":len(relsDataIndex[id]["childs"])} for id in lvl1]
props = [{k:t.get(k, "missing") for k in getKeys} for t in props]
plot(props)

,id,name,official_name,parent_id,ISO3166-2,admin_level,childs_num
0,28322,Mecklenburg-Vorpommern,missing,51477,DE-MV,4,8
1,51529,Schleswig-Holstein,missing,51477,DE-SH,4,15
2,62341,Rheinland-Pfalz,missing,51477,DE-RP,4,36
3,62366,Thüringen,Freistaat Thüringen,51477,DE-TH,4,22
4,62372,Saarland,missing,51477,DE-SL,4,6
5,62422,Berlin,Berlin,51477,DE-BE,4,0
6,62467,Sachsen,Freistaat Sachsen,51477,DE-SN,4,13
7,62504,Brandenburg,missing,51477,DE-BB,4,18
8,62607,Sachsen-Anhalt,missing,51477,DE-ST,4,14
9,62611,Baden-Württemberg,missing,51477,DE-BW,4,44


### lvl 2

In [118]:
props = [{**dictRemoveKeys(relsDataIndex[id],["childs", "tags"]), **relsDataIndex[id]["tags"], "childs_num":len(relsDataIndex[id]["childs"])} for id in relsDataIndex["51529"]["childs"]]
props = [{k:t.get(k, "missing") for k in getKeys} for t in props]
plot(props)

,id,name,official_name,parent_id,ISO3166-2,admin_level,childs_num
0,27014,Schleswig-Flensburg,Kreis Schleswig-Flensburg,51529,missing,6,123
1,27016,Kreis Steinburg,Kreis Steinburg,51529,missing,6,111
2,27017,Rendsburg-Eckernförde,Kreis Rendsburg-Eckernförde,51529,missing,6,165
3,27019,Nordfriesland,Kreis Nordfriesland,51529,missing,6,129
4,27020,Flensburg,missing,51529,missing,6,0
5,27021,Kiel,Kreisfreie Stadt Kiel,51529,missing,6,0
6,27025,Ostholstein,Kreis Ostholstein,51529,missing,6,36
7,27026,Plön,Kreis Plön,51529,missing,6,85
8,27027,Lübeck,Kreisfreie Stadt Lübeck,51529,missing,6,0
9,27028,Dithmarschen,Kreis Dithmarschen,51529,missing,6,116


### lvl 3

In [119]:
props = [{**dictRemoveKeys(relsDataIndex[id],["childs", "tags"]), **relsDataIndex[id]["tags"], "childs_num":len(relsDataIndex[id]["childs"])} for id in relsDataIndex["62546"]["childs"]]
props = [{k:t.get(k, "missing") for k in getKeys} for t in props]
plot(props)

,id,name,official_name,parent_id,ISO3166-2,admin_level,childs_num
0,312038,Ammersbek,missing,62546,missing,8,0
1,335683,Reinbek,missing,62546,missing,8,0
2,419144,Trittau,missing,62546,missing,8,0
3,444300,Tangstedt,missing,62546,missing,8,0
4,453723,Bargteheide,missing,62546,missing,8,0
5,453724,Jersbek,missing,62546,missing,8,0
6,453725,Bargfeld-Stegen,missing,62546,missing,8,0
7,453726,Nienwohld,missing,62546,missing,8,0
8,453727,Elmenhorst,missing,62546,missing,8,0
9,453743,Tremsbüttel,missing,62546,missing,8,0


# derive data

### [OLD] make data tree

In [12]:
base = list(map(lambda x: load()(x), glob.glob("data/*/addsDataMinTree.json")))

In [ ]:
# addsDataTree = makeAddsTree([countryID], {k:dictFilterKeys(v, keysMin) for k, v in relTagsIndex.items()})
# len(addsDataTree[countryID]["childs"])

In [76]:
dump(os.path.join("data", "addsDataMinJoinedTree.json"), addsDataMinJoinedTree)

### [Dev]

In [11]:
ids = ['2','3']
index = {'2':['a','b'],3:['c'],'a':['d']}
dataIndex = {'2':{'tags':{'name':'name-2'}},
             '3':{'tags':{'name':'name-3'}},
             'a':{'tags':{'name':'name-a'}},
             'b':{'tags':{'name':'name-b'}},
             'c':{'tags':{'name':'name-c'}},
             'd':{'tags':{'name':'name-d'}}}

In [ ]:
htmlTree = makeHTMLTree(['2'], index, dataIndex)
print(htmlTree)

### [OLD] make tree for each country

In [15]:
htmlTree = makeHTMLTree([cntrId], childsIndex, relsDataIndex)
# print(htmlTree)

In [16]:
import re
htmlTree = re.sub(r'^<ul>', '', htmlTree)
htmlTree = re.sub(r'</ul>$', '', htmlTree)

In [109]:
soup = BeautifulSoup(htmlTree, 'html.parser')
pretty_html = soup.prettify()
dump(treeOutputDir, pretty_html)

In [69]:
importlib.reload(toolsGeneral.main)
importlib.reload(toolsOSM.overpass)
from toolsGeneral.main import (load, dump)
from toolsOSM.overpass import (makeTree, makeHTMLTree)

### make countries trees and join

In [127]:
countrs = list(filter(lambda x: re.match(r'\d+', x), os.listdir("./data")))
print(countrs)

['1428125', '167454', '2202162', '288247', '51477']


In [133]:
idChildsDataIndex = list(map(lambda x: [x,load(f"./data/{x}/childsIndex.json"), load(f"./data/{x}/relsDataIndex.json")], countrs))

In [134]:
countrsTrees = list(map(lambda x: makeHTMLTree([x[0]], x[1], x[2]), idChildsDataIndex))

KeyError: '47826'

In [14]:
dir = {'1':'a', '2': 'b'}
value = next((dir[key] for key in ['3','2'] if key in dir), None)
print(value)

b


In [104]:
next((dir[key] for key in ['4','2'] if key in dir))

'b'

In [8]:
def formattify(tree):
    formatted = re.sub(r'^<ul>', '', tree)
    formatted = re.sub(r'</ul>$', '', formatted)
    soup = BeautifulSoup(formatted, 'html.parser')
    formatted = soup.prettify()
    return formatted

formattedTree = "<div id=\"addSelectionTree\"><ul>" + ''.join(list(map(formattify, countrsTrees))) + "</ul></div>"

### insert tree into html page

In [9]:
soup = BeautifulSoup(load(r"c:\Users\gonta\D\study sync\full stack\projects\administrative divisions query\data\indexBase.html"), "html.parser")

In [10]:
forReplace = BeautifulSoup(formattedTree, 'html.parser')
toReplace  = soup.find(id="addSelectionTree")
toReplace.replace_with(forReplace)

<div id="addSelectionTree">
<!-- insert html tree here -->
</div>

In [11]:
dump('./data/index.html', str(soup))